In [1]:
!ls


app.py				      logs
back_filled_forecast.ipynb	      model_params_logs.csv
cict_demo.py			      model_utils.py
daily_US_All_2020-12-20.csv	      pwlf_mod.py
data				      __pycache__
data_processed			      SEIR.py
forecast_utils.py		      template
format_forecast_ouput.ipynb	      Try_plotly.ipynb
generate_US_formatted_forecast.ipynb  Try_SEIR.ipynb
__init__.py


In [7]:
import pandas as pd
import epiweeks
import datetime as dt
import numpy as np

In [3]:
import model_utils as mu

In [4]:
mu.DEATH_RATE = 0.36
mu.ICU_RATE = 0.78
mu.HOSPITAL_RATE = 2.18
mu.SYMPTOM_RATE = 10.2
mu.INFECT_2_HOSPITAL_TIME = 11
mu.HOSPITAL_2_ICU_TIME = 4
mu.ICU_2_DEATH_TIME = 4
mu.ICU_2_RECOVER_TIME = 7
mu.NOT_ICU_DISCHARGE_TIME = 5

In [6]:
local_death_data = mu.get_data_by_country('US', 'All', type='deaths')

In [9]:
policy_change_dates = ["2020-03-19", "2020-06-12", "2020-07-13", "2020-10-20", "2020-12-03"]

In [10]:
policy_effective_dates = pd.to_datetime(policy_change_dates) + dt.timedelta(
        mu.INFECT_2_HOSPITAL_TIME + mu.HOSPITAL_2_ICU_TIME + mu.ICU_2_DEATH_TIME)
daily_local_death_new = mu.get_daily_data(local_death_data)
smoothing_days = 7
daily_local_death_avg = daily_local_death_new.rolling(smoothing_days, min_periods=1).mean()
# Because of this smoothing step, we need to time var of prediction by smoothing_days=3.
# Rolling set the label at the right edge of the windows, so we need to blank out the first 6 days after
# policy effective dates since it mixes before and after change curve

#shift ahead 1 day to avoid overfitted due to average of exponential value
#daily_local_death_new = daily_local_death_new.shift(1)
daily_local_death_avg.columns = ['death']
log_daily_death = np.log(daily_local_death_avg)

/home/quoc/anaconda3/envs/st65/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]


In [13]:
policy_effective_date = policy_effective_dates[0]

In [16]:
policy_effective_date

Timestamp('2020-04-07 00:00:00')

In [25]:
for policy_effective_date in policy_effective_dates:
    daily_local_death_avg = daily_local_death_avg.loc[
        (daily_local_death_avg.index>=policy_effective_date+dt.timedelta(7))|
        (daily_local_death_avg.index<policy_effective_date)]

In [27]:
daily_local_death_avg.tail(60)

,death
2020-10-22,786.571429
2020-10-23,793.714286
2020-10-24,821.571429
2020-10-25,813.571429
2020-10-26,817.000000
2020-10-27,823.571429
2020-10-28,801.142857
2020-10-29,815.857143
2020-10-30,827.142857
2020-10-31,819.000000


In [24]:
daily_local_death_avg.loc[(daily_local_death_avg.index>=policy_effective_date+dt.timedelta(7))|
                          (daily_local_death_avg.index<policy_effective_date)].head(60)

,death
2020-02-28,0.000000
2020-02-29,0.500000
2020-03-01,0.333333
2020-03-02,1.500000
2020-03-03,1.400000
2020-03-04,1.833333
2020-03-05,1.714286
2020-03-06,2.000000
2020-03-07,2.285714
2020-03-08,2.857143


In [18]:
fips = pd.read_csv('../../../github/covid19-forecast-hub/data-locations/locations.csv')

In [4]:
input_forecast = pd.read_csv("daily_US_All_2020-12-20.csv")

In [5]:
input_forecast.tail(n=70)

,Unnamed: 0,death,predicted_death,lower_bound,upper_bound,infected,symptomatic,hospitalized,hospital_beds,ICU,confirmed
305,2020-12-10,2920.0,2453.887964,2167.678644,2777.886914,1.177338e+06,87516.272517,18704.458244,122447.724167,30326.885178,226684.0
306,2020-12-11,3283.0,2525.496563,2221.185495,2871.499433,1.211695e+06,90070.145287,19250.285953,126020.955744,31211.874953,234103.0
307,2020-12-12,2354.0,2599.194822,2275.810933,2968.530303,1.247054e+06,92698.544381,19812.041838,129698.460258,32122.690224,219193.0
308,2020-12-13,1389.0,2675.043720,2331.592850,3069.085970,1.283445e+06,95403.644603,20390.190709,133483.280571,33060.084625,191142.0
309,2020-12-14,1484.0,2753.106018,2388.568935,3173.277788,1.320898e+06,98187.684223,20985.210942,137378.548338,34024.833779,192846.0
...,...,...,...,...,...,...,...,...,...,...,...
370,2021-02-13,NaN,15916.151557,9998.610245,25335.909108,NaN,NaN,NaN,NaN,196703.071963,NaN
371,2021-02-14,NaN,16380.611764,10234.063765,26218.758054,NaN,NaN,NaN,NaN,NaN,NaN
372,2021-02-15,NaN,16858.625705,10475.042982,27132.419519,NaN,NaN,NaN,NaN,NaN,NaN
373,2021-02-16,NaN,17350.588901,10721.677791,28077.968865,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
input_forecast.rename(columns={'Unnamed: 0': 'date'}, inplace = True)

In [203]:
abbr = 'US'
forecast_date = '2020-12-21'
target_metric = 'death'
target_aggr = 'inc'

In [11]:
metric_map ={'death': 'predicted_death'}

In [9]:
def get_epiweek_enddate(x):
    return epiweeks.Week.fromdate(pd.to_datetime(x).date()).enddate()

In [10]:
def get_target_str(target_end_date):
    forecast_date_week_end = get_epiweek_enddate(forecast_date)
    target = '{week} wk ahead {target_aggr} {target_metric}'.format(week=(target_end_date - forecast_date_week_end).days//7 + 1,
                                                               target_aggr=target_aggr,
                                                               target_metric=target_metric)
    return target

In [73]:
def format_forecast(input_forecast, 
                    location_name, 
                    forecast_date,
                    target_metric,
                    target_aggr):
    forecast_date = pd.to_datetime(forecast_date).date()
    input_forecast['target_end_date'] = input_forecast.date.apply(get_epiweek_enddate)
    input_forecast['target'] = input_forecast.target_end_date.apply(get_target_str)
    input_forecast['forecast_date'] = forecast_date
    input_forecast['location'] = fips.query('location_name == @location_name').location.iloc[0]
    input_forecast['quantile'] = 'NA'
    input_forecast['type'] = 'point'
    input_forecast.rename(columns={metric_map[target_metric]:'value'}, inplace=True)
    output = input_forecast[['forecast_date', 'target', 'target_end_date', 'quantile', 'type', 'value', 'location']]
    return output.groupby(['forecast_date', 'target', 'target_end_date', 'quantile', 'type', 'location']).sum().reset_index().query('target_end_date>forecast_date')

In [54]:
format_forecast(input_forecast)

,forecast_date,target,target_end_date,quantile,type,location,value
45,2020-12-20,1 wk ahead inc death,2020-12-26,NA,point,US,23423.635211
46,2020-12-20,2 wk ahead inc death,2021-01-02,NA,point,US,28318.467783
47,2020-12-20,3 wk ahead inc death,2021-01-09,NA,point,US,34236.172582
48,2020-12-20,4 wk ahead inc death,2021-01-16,NA,point,US,41390.499022
49,2020-12-20,5 wk ahead inc death,2021-01-23,NA,point,US,50039.863689
50,2020-12-20,6 wk ahead inc death,2021-01-30,NA,point,US,60496.684438
51,2020-12-20,7 wk ahead inc death,2021-02-06,NA,point,US,73138.665019
52,2020-12-20,8 wk ahead inc death,2021-02-13,NA,point,US,88422.437866
53,2020-12-20,9 wk ahead inc death,2021-02-20,NA,point,US,74260.390986


In [74]:
def generate_formatted_forecast(scope,
                                location_name,
                                forecast_date,
                                target_metric = 'death',
                                target_aggr = 'inc'):
    forecast_date = pd.to_datetime(forecast_date).date()
    if scope == 'World':
        forecast_fun = mu.get_metrics_by_country
        policy_date_fun = mu.get_policy_change_dates_by_country
    else:
        forecast_fun = mu.get_metrics_by_state_US
        policy_date_fun = mu.get_policy_change_dates_by_state_US
    input_forecast, _, _ = forecast_fun(location_name, 
                                        forecast_horizon=60,
                                        policy_change_dates=policy_date_fun(location_name),
                                        back_test=True, last_data_date=forecast_date)
    input_forecast.index.rename('date', inplace=True)
    input_forecast.reset_index(inplace=True)
    return format_forecast(input_forecast, location_name, forecast_date, target_metric, target_aggr)

In [185]:
US_forecast = generate_formatted_forecast('World', 'US', '2020-12-20')

/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: divide by zero encountered in log
  log_daily_death = np.log(daily_local_death_new)


In [186]:
US_forecast = US_forecast.query('target!="9 wk ahead inc death"')

In [84]:
CA = generate_formatted_forecast('US', 'California', '2020-12-20')

/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: divide by zero encountered in log
  log_daily_death = np.log(daily_local_death_new)
/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: invalid value encountered in log
  log_daily_death = np.log(daily_local_death_new)


In [194]:
US_forecast

,forecast_date,target,target_end_date,quantile,type,location,value
45,2020-12-20,1 wk ahead inc death,2020-12-26,NA,point,US,23423.635211
46,2020-12-20,2 wk ahead inc death,2021-01-02,NA,point,US,28318.467783
47,2020-12-20,3 wk ahead inc death,2021-01-09,NA,point,US,34236.172582
48,2020-12-20,4 wk ahead inc death,2021-01-16,NA,point,US,41390.499022
49,2020-12-20,5 wk ahead inc death,2021-01-23,NA,point,US,50039.863689
50,2020-12-20,6 wk ahead inc death,2021-01-30,NA,point,US,60496.684438
51,2020-12-20,7 wk ahead inc death,2021-02-06,NA,point,US,73138.665019
52,2020-12-20,8 wk ahead inc death,2021-02-13,NA,point,US,88422.437866
45,2020-12-20,1 wk ahead cum death,2020-12-26,NA,point,US,339582.635211
46,2020-12-20,2 wk ahead cum death,2021-01-02,NA,point,US,367901.102995


In [188]:
def add_cum_forecast(inc_forecast, last_epi_week_cum):
    cum_forecast = inc_forecast.copy()
    cum_forecast['value'] = cum_forecast.value.cumsum()+last_epi_week_cum
    cum_forecast['target'] = cum_forecast.target.str.replace('inc', 'cum')
    return pd.concat([inc_forecast, cum_forecast])

In [189]:
last_epiweek_enddate = get_epiweek_enddate(forecast_date+epiweeks.timedelta(-7))

In [190]:
latest_cum_US = mu.get_data_by_country('US').loc[last_epiweek_enddate][0]

In [191]:
US_forecast = add_cum_forecast(US_forecast, latest_cum_US)

In [192]:
latest_cum_US

316159.0

In [193]:
US_state_list = mu.get_data(scope='US', type='deaths').State.unique()

In [195]:
for state in US_state_list:
    try:
        print(state)
        state_forecast = generate_formatted_forecast('US', state, '2020-12-20').query('target!="9 wk ahead inc death"')
        latest_cum_state = mu.get_data_by_state(state).loc[last_epiweek_enddate][0]
        state_forecast = add_cum_forecast(state_forecast, latest_cum_state)
        US_forecast = pd.concat([US_forecast, state_forecast])
    except (ValueError, IndexError):
        pass

Alabama


/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: divide by zero encountered in log
  log_daily_death = np.log(daily_local_death_new)
/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: invalid value encountered in log
  log_daily_death = np.log(daily_local_death_new)


Alaska


/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: divide by zero encountered in log
  log_daily_death = np.log(daily_local_death_new)


American Samoa
Arizona


...

/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: divide by zero encountered in log
  log_daily_death = np.log(daily_local_death_new)


Wisconsin


/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: divide by zero encountered in log
  log_daily_death = np.log(daily_local_death_new)


Wyoming


/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: divide by zero encountered in log
  log_daily_death = np.log(daily_local_death_new)
/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: invalid value encountered in log
  log_daily_death = np.log(daily_local_death_new)


In [198]:
US_forecast.tail(60)

,forecast_date,target,target_end_date,quantile,type,location,value
49,2020-12-20,5 wk ahead inc death,2021-01-23,NA,point,53,92.768179
50,2020-12-20,6 wk ahead inc death,2021-01-30,NA,point,53,95.133051
51,2020-12-20,7 wk ahead inc death,2021-02-06,NA,point,53,97.558209
52,2020-12-20,8 wk ahead inc death,2021-02-13,NA,point,53,100.045190
45,2020-12-20,1 wk ahead cum death,2020-12-26,NA,point,53,3187.882131
46,2020-12-20,2 wk ahead cum death,2021-01-02,NA,point,53,3273.902609
47,2020-12-20,3 wk ahead cum death,2021-01-09,NA,point,53,3362.115945
48,2020-12-20,4 wk ahead cum death,2021-01-16,NA,point,53,3452.578039
49,2020-12-20,5 wk ahead cum death,2021-01-23,NA,point,53,3545.346218
50,2020-12-20,6 wk ahead cum death,2021-01-30,NA,point,53,3640.479269


In [200]:
US_forecast.query('target=="9 wk ahead cum death"')

,forecast_date,target,target_end_date,quantile,type,location,value


In [202]:
US_forecast.to_csv('{}-AIpert-pwllnod.csv'.format(forecast_date), index=False)

In [55]:
scope = 'World'
location_name = 'US'

In [56]:
forecast_date = pd.to_datetime(forecast_date).date()

In [57]:
if scope == 'World':
    forecast_fun = mu.get_metrics_by_country
    policy_date_fun = mu.get_policy_change_dates_by_country
else:
    forecast_fun = mu.get_metrics_by_state_US
    policy_date_fun = mu.get_policy_change_dates_by_state_US

In [58]:
input_forecast, _, _ = forecast_fun(location_name, 
                                    forecast_horizon=60,
                                    policy_change_dates=policy_date_fun(location_name),
                                    back_test=True, last_data_date=forecast_date)

/home/quoc/bitbucket/covid-19/prognosis/model_utils.py:247: RuntimeWarning: divide by zero encountered in log
  log_daily_death = np.log(daily_local_death_new)


In [59]:
input_forecast.index.rename('date', inplace=True)

In [60]:
input_forecast.reset_index(inplace=True)

In [62]:
format_forecast(input_forecast).query('target!="9 wk ahead inc death"')

,forecast_date,target,target_end_date,quantile,type,location,value
45,2020-12-20,1 wk ahead inc death,2020-12-26,NA,point,US,23423.635211
46,2020-12-20,2 wk ahead inc death,2021-01-02,NA,point,US,28318.467783
47,2020-12-20,3 wk ahead inc death,2021-01-09,NA,point,US,34236.172582
48,2020-12-20,4 wk ahead inc death,2021-01-16,NA,point,US,41390.499022
49,2020-12-20,5 wk ahead inc death,2021-01-23,NA,point,US,50039.863689
50,2020-12-20,6 wk ahead inc death,2021-01-30,NA,point,US,60496.684438
51,2020-12-20,7 wk ahead inc death,2021-02-06,NA,point,US,73138.665019
52,2020-12-20,8 wk ahead inc death,2021-02-13,NA,point,US,88422.437866


In [77]:
forecast_date = pd.to_datetime(forecast_date).date()

In [88]:
forecast_date_week_end = get_epiweek_enddate(forecast_date)

In [79]:
target_end_date

datetime.date(2020, 12, 26)

In [89]:
(target_end_date - forecast_date_week_end).days//7 + 1

1

In [90]:
target = '{week} wk ahead {target_aggr} {target_metric}'.format(week=(target_end_date - forecast_date_week_end).days//7 + 1,
                                                               target_aggr=target_aggr,
                                                               target_metric=target_metric)

In [91]:
target

'1 wk ahead inc death'

In [94]:
input_forecast['target_end_date'] = input_forecast.date.apply(get_epiweek_enddate)

In [96]:
input_forecast['target'] = input_forecast.target_end_date.apply(get_target_str)

In [98]:
input_forecast['forecast_date'] = forecast_date

In [99]:
input_forecast['location'] = fips.query('abbreviation == @abbr').location[0]

In [100]:
input_forecast['quantile'] = 'NA'

In [102]:
input_forecast['type'] = 'point'

In [105]:
input_forecast.rename(columns={metric_map[target_metric]:'value'}, inplace=True)

In [109]:
output = input_forecast[['forecast_date', 'target', 'target_end_date', 'quantile', 'type', 'value', 'location']]

In [112]:
output.groupby(['forecast_date', 'target', 'target_end_date', 'quantile', 'type', 'location']).sum().reset_index().query('target_end_date>forecast_date')

,forecast_date,target,target_end_date,quantile,type,location,value
45,2020-12-20,1 wk ahead inc death,2020-12-26,NA,point,US,25007.359242
46,2020-12-20,2 wk ahead inc death,2021-01-02,NA,point,US,30585.261090
47,2020-12-20,3 wk ahead inc death,2021-01-09,NA,point,US,37407.316258
48,2020-12-20,4 wk ahead inc death,2021-01-16,NA,point,US,45751.033661
49,2020-12-20,5 wk ahead inc death,2021-01-23,NA,point,US,55955.820690
50,2020-12-20,6 wk ahead inc death,2021-01-30,NA,point,US,68436.789697
51,2020-12-20,7 wk ahead inc death,2021-02-06,NA,point,US,83701.644016
52,2020-12-20,8 wk ahead inc death,2021-02-13,NA,point,US,102371.330421
53,2020-12-20,9 wk ahead inc death,2021-02-20,NA,point,US,68446.734785


In [31]:
import epiweeks

In [8]:
dir(pymmwr)

['Epiweek',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_start_date_of_year',
 'attr',
 'date_to_epiweek',
 'datetime',
 'epiweek_to_date',
 'epiweeks_in_year']

In [45]:
dir(epiweeks)

['Iterator',
 'Tuple',
 'Week',
 'Year',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_check_system',
 '_check_week',
 '_check_year',
 '_system_adjustment',
 '_year_start',
 '_year_total_weeks',
 'date',
 'timedelta']

In [50]:
epiweeks.Week.fromdate(pd.to_datetime(input_forecast.date[0]).date()).enddate()

datetime.date(2020, 2, 15)

In [55]:
epiweeks.Week.fromdate(pd.to_datetime(input_forecast.date[0]).date())

Week(2020, 7, CDC)

In [118]:
import datetime as dt

In [29]:
epd = pymmwr.date_to_epiweek(pd.to_datetime(input_forecast['Unnamed: 0'][0]).date())

In [42]:
epd

Epiweek(year=2020, week=7, day=1)

In [44]:
epd.year

2020

In [21]:
dt.datetime.strptime(input_forecast['Unnamed: 0'][374], '%Y-%m-%d')

datetime.datetime(2021, 2, 17, 0, 0)

In [27]:
dt.datetime.strptime(input_forecast['Unnamed: 0'][374], '%Y-%m-%d').date()

datetime.date(2021, 2, 17)

In [32]:
fips = pd.read_csv('../../../github/covid19-forecast-hub/data-locations/locations.csv')

In [33]:
fips.head()

,abbreviation,location,location_name,population
0,US,US,US,328728466.0
1,AL,01,Alabama,4903185.0
2,AK,02,Alaska,731545.0
3,AZ,04,Arizona,7278717.0
4,AR,05,Arkansas,3017804.0


In [41]:
fips.query('abbreviation == @abbr').location[0]

'US'

In [72]:
input_forecast['target_end_date'] = input_forecast.date.apply(get_epiweek_enddate)

In [97]:
input_forecast.tail(n=70)

,date,death,predicted_death,lower_bound,upper_bound,infected,symptomatic,hospitalized,hospital_beds,ICU,confirmed,target_end_date,target
305,2020-12-10,2920.0,2453.887964,2167.678644,2777.886914,1.177338e+06,87516.272517,18704.458244,122447.724167,30326.885178,226684.0,2020-12-12,-1 wk ahead inc death
306,2020-12-11,3283.0,2525.496563,2221.185495,2871.499433,1.211695e+06,90070.145287,19250.285953,126020.955744,31211.874953,234103.0,2020-12-12,-1 wk ahead inc death
307,2020-12-12,2354.0,2599.194822,2275.810933,2968.530303,1.247054e+06,92698.544381,19812.041838,129698.460258,32122.690224,219193.0,2020-12-12,-1 wk ahead inc death
308,2020-12-13,1389.0,2675.043720,2331.592850,3069.085970,1.283445e+06,95403.644603,20390.190709,133483.280571,33060.084625,191142.0,2020-12-19,0 wk ahead inc death
309,2020-12-14,1484.0,2753.106018,2388.568935,3173.277788,1.320898e+06,98187.684223,20985.210942,137378.548338,34024.833779,192846.0,2020-12-19,0 wk ahead inc death
...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,2021-02-13,NaN,15916.151557,9998.610245,25335.909108,NaN,NaN,NaN,NaN,196703.071963,NaN,2021-02-13,8 wk ahead inc death
371,2021-02-14,NaN,16380.611764,10234.063765,26218.758054,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-20,9 wk ahead inc death
372,2021-02-15,NaN,16858.625705,10475.042982,27132.419519,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-20,9 wk ahead inc death
373,2021-02-16,NaN,17350.588901,10721.677791,28077.968865,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-20,9 wk ahead inc death
